# Controller Prober

Controller is an Arduino Nano clone on a custom PCB with 2 stepper drivers (X & Y).

Instructions only show a Windows only program for 'engraving' pictures.

Goal: Figure out what the board is running & how to use it with Python.

In [1]:
import serial
from time import sleep

## 1. Baud Rate

How fast are we talking?

In [13]:
baudrates = [1200, 2400, 4800, 9600, 19200, 38400, 57600, 115200]

In [14]:
for baudrate in baudrates:
    with serial.Serial(
        port='/dev/ttyUSB0',
        baudrate=baudrate,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        timeout=1,
        xonxoff=0,
        rtscts=0
    ) as arduino:
        # Toggle DTR to reset Arduino
        arduino.setDTR(False)
        sleep(2)
        print("{}: {}".format(baudrate, arduino.read_all()))

1200: b'\xfa'
2400: b'\x95'
4800: b'\x87\xf3'
9600: b'\xb6|j\xf2'
19200: b'\x101A\xd2\xd2\x93\xff'
38400: b'\xd4\x9a\x98\x08b\xc6\x93\xc5\x80\xb0\xafP\xb9\xf3'
57600: b'\xbc\x8d\x0e\x9d!!9L\x84\rJ\xbdH\x0e\x8a\xd8\x9f\xee\x8c\xcc'
115200: b'benbox laser 3.1(c) mul 2015.06.06\r\nstart\r\nok\r\n'


## 2. Dialect

What are we talking?

In [6]:
try:
    # If we're already executed this cell before, close the serial port first.
    ser.close()
except:
    pass

cfg = {
    "port": '/dev/ttyUSB0',
    "baudrate": 115200,
    "bytesize": serial.EIGHTBITS,
    "parity": serial.PARITY_NONE,
    "stopbits": serial.STOPBITS_ONE,
    "timeout": 10,
    "xonxoff": 0,
    "rtscts": 0,
}
ser = serial.Serial(**cfg)
ser.setDTR(False)
sleep(2)
ser.read_all()

b'benbox laser 3.1(c) mul 2015.06.06\r\nstart\r\nok\r\n'

In [7]:
ser.write(b"\n\n")
ser.read_all()

b''

In [8]:
ser.write(b"$$\n")
ser.read_all()

b''

In [9]:
ser.write(b"$$\r\n")
ser.read_all()

b'ok\r\n'

In [10]:
ser.write(b"help\r\n")
ser.read_all()

b'ok\r\n'

In [11]:
ser.write(b"help\r\n")
sleep(5)
ser.read_all()

b'ok\r\nok\r\n'